<a href="https://colab.research.google.com/github/manthan-mehta/ImageClassification/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [0]:
def load_cifar10(cifar10_folder_path,batch_id):
  import pickle
  with open(cifar10_folder_path + '/data_batch_' +str(batch_id),'rb') as fo:
    dict = pickle.load(fo,encoding='latin1')
  return dict

In [0]:
train = None
Y_train = []
for i in range(1,6):
  dict1 = load_cifar10('/content',1)
  if i==1:
    train = dict1['data']
  else:
    train = np.vstack((train,dict1['data']))
  Y_train += dict1['labels']

In [0]:
X_train = train.reshape((len(train), 3, 32, 32)).transpose(0, 2, 3, 1)
Y_train = np.asarray(Y_train)

In [0]:
import pickle
with open('/content/test_batch','rb') as fo:
    test_data = pickle.load(fo,encoding='latin1')
X_test = test_data['data'].reshape((len(test_data['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
Y_test = np.asarray(test_data['labels'])

In [71]:
# Split the data into train, val, and test sets. In addition we will
# create a small development set as a subset of the training data;
# we can use this for development so our code runs faster.
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

# Our validation set will be num_validation points from the original
# training set.
mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
Y_val = Y_train[mask]

# Our training set will be the first num_train points from the original
# training set.
mask = range(num_training)
X_train = X_train[mask]
Y_train = Y_train[mask]

# We will also make a development set, which is a small subset of
# the training set.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
Y_dev = Y_train[mask]

# We use the first num_test points of the original test set as our
# test set.
mask = range(num_test)
X_test = X_test[mask]
Y_test = Y_test[mask]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', Y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', Y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', Y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (1000, 32, 32, 3)
Test labels shape:  (1000,)


In [72]:
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

# print out the shapes of the data
print('Training data shape: ', X_train.shape)
print('Validation data shape: ', X_val.shape)
print('Test data shape: ', X_test.shape)
print('dev data shape: ', X_dev.shape)

Training data shape:  (49000, 3072)
Validation data shape:  (1000, 3072)
Test data shape:  (1000, 3072)
dev data shape:  (500, 3072)


In [0]:
mean_image = np.mean(X_train,axis=0)
mean_image = mean_image.astype('uint8')

In [0]:
X_train -= mean_image
X_dev -= mean_image
X_test -= mean_image
X_val -= mean_image

In [81]:
#Add Bias 
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

(49000, 3073) (1000, 3073) (1000, 3073) (500, 3073)


In [0]:
# generate a random SVM weight matrix of small numbers
W = np.random.randn(3073, 10) * 0.0001 